In [1]:
import pandas as pd
import numpy as np

In [2]:
crimeData = pd.read_csv('./Crime_Data_from_2010_to_Present.csv')
moCodes = pd.read_csv('./MO_Codes.csv')
poi = pd.read_excel('./Map_of_Locations_Points_of_Interest_Children.xlsx')
print('Crime data shape: ', crimeData.shape)
print('MO Codes data shape: ', moCodes.shape)
print('Points of interest data shape: ', poi.shape)

Crime data shape:  (1700816, 26)
MO Codes data shape:  (532, 2)
Points of interest data shape:  (64974, 7)


In [14]:
crimeData.columns

Index(['ID', 'Date Reported', 'Date Occurred', 'Time Occurred', 'Area ID',
       'Area Name', 'Reporting District', 'Crime Code',
       'Crime Code Description', 'Victim Age', 'Victim Sex', 'Victim Descent',
       'Premise Code', 'Premise Description', 'Weapon Used Code',
       'Weapon Description', 'Status Code', 'Status Description',
       'Crime Code 1', 'Crime Code 2', 'Crime Code 3', 'Crime Code 4',
       'Address', 'Cross Street', 'lat', 'lon', 'type', 'youthVictim',
       'Time Occurred Hour'],
      dtype='object')

# Data Processing
For Crime_Data_2010_2017.csv:
        
        - use DR number as the unique identifier per row
        - unfold MO Codes column & merge in MO_Codes data (Note: MO code = 1217, 1257, 1258, 1259 in addition to victim age to determine crimes where victim was child)
        -split Location into latitude and longitude

For Map_of_Locations_Points_of_Interest_Children.xlsx:
        
        - union the data with crime data and designate using a column if it is associated with crime data or with POI

### MO Codes per Crime data -> MOCodeCrime.csv

In [3]:
drNumber = list(crimeData['DR Number'])
codes = list(crimeData['MO Codes'])
drNumberList = []
codeList = [] 
for i in range(len(codes)):
    try:
        code = codes[i].split()
        for j in code:
            codeList.append(j)
            drNumberList.append(drNumber[i])
    except:
        pass

In [4]:
moCodesCrime = pd.DataFrame()
moCodesCrime['DR Number'] = drNumberList
moCodesCrime['MO_Code'] = codeList
moCodesCrime = moCodesCrime.astype(int)
MOCodeCrime = pd.merge(moCodesCrime, moCodes)
MOCodeCrime = MOCodeCrime.rename(columns={'DR Number': 'ID'})
MOCodeCrime.to_csv('./TableauReady/MOCodeCrime.csv', index=False)

### Union Crime and POI data together -> CrimePOI.csv

In [5]:
locations = list(crimeData['Location '])
lat = []
lon = []
for l in locations:
    try:
        lat.append(np.float(l.split()[0].rstrip(',').lstrip('(')))
        lon.append(np.float(l.split()[1].rstrip(')')))
    except:
        lat.append(None)
        lon.append(None)
crimeData['lat'] = lat
crimeData['lon'] = lon
crimeData['type'] = 'crime'
crimeData = crimeData.drop(['MO Codes', 'Location '], axis=1) #Delete not usable columns
crimeData['DR Number'] = crimeData['DR Number'].astype(int)
crimeData = crimeData.rename(columns={'DR Number': 'ID'})
crimeData.head()

,ID,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,Victim Age,...,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,lat,lon,type
0,1208575,03/14/2013,03/11/2013,1800,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,30.0,...,Adult Other,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,33.9829,-118.3338,crime
1,102005556,01/25/2010,01/22/2010,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,34.0454,-118.3157,crime
2,418,03/19/2013,03/18/2013,2030,18,Southeast,1823,510,VEHICLE - STOLEN,12.0,...,Invest Cont,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,33.9420,-118.2717,crime
3,101822289,11/11/2010,11/10/2010,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,33.9572,-118.2717,crime
4,42104479,01/11/2014,01/04/2014,2300,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),84.0,...,Invest Cont,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,34.2009,-118.6369,crime


In [6]:
# Get IDs where MO code = 1217, 1257, 1258, 1259 in addition to victim age to determine crimes where victim was child
crimeVictims1 = list(MOCodeCrime['ID'][MOCodeCrime['MO_Code'].isin([1217, 1257, 1258, 1259])].unique())
crimeVictims2 = list(crimeData['ID'][crimeData['Victim Age'] <= 18].unique())
crimeVictims = list(set(crimeVictims1)|set(crimeVictims2))
youthVictimsData = crimeData[crimeData['ID'].isin(crimeVictims)]
youthVictimsData = youthVictimsData.drop(youthVictimsData[youthVictimsData['Victim Age'] > 18].index)
youthVictims = list(youthVictimsData['ID'])
print('Number of incidences against youth: ', len(youthVictims))

crimeData['youthVictim'] = np.where(crimeData['ID'].isin(youthVictims), 1, 0)

Number of incidences against youth:  268848


In [7]:
# Convert Time Occurred to Time Occurred Hour
def toHour(row):
    if row['Time Occurred'] <= 59:
        return 0
    elif row['Time Occurred'] <= 159:
        return 1
    elif row['Time Occurred'] <= 259:
        return 2
    elif row['Time Occurred'] <= 359:
        return 3
    elif row['Time Occurred'] <= 459:
        return 4
    elif row['Time Occurred'] <= 559:
        return 5
    elif row['Time Occurred'] <= 659:
        return 6
    elif row['Time Occurred'] <= 759:
        return 7
    elif row['Time Occurred'] <= 859:
        return 8
    elif row['Time Occurred'] <= 959:
        return 9
    elif row['Time Occurred'] <= 1059:
        return 10
    elif row['Time Occurred'] <= 1159:
        return 11
    elif row['Time Occurred'] <= 1259:
        return 12
    elif row['Time Occurred'] <= 1359:
        return 13
    elif row['Time Occurred'] <= 1459:
        return 14
    elif row['Time Occurred'] <= 1559:
        return 15
    elif row['Time Occurred'] <= 1659:
        return 16
    elif row['Time Occurred'] <= 1759:
        return 17
    elif row['Time Occurred'] <= 1859:
        return 18
    elif row['Time Occurred'] <= 1959:
        return 19
    elif row['Time Occurred'] <= 2059:
        return 20
    elif row['Time Occurred'] <= 2159:
        return 21
    elif row['Time Occurred'] <= 2259:
        return 22
    elif row['Time Occurred'] <= 2359:
        return 23
    else:
        return 24
    
crimeData['Time Occurred Hour'] = crimeData.apply(toHour, axis=1)

In [8]:
# Changing the abbreviations to the whole description
Victims_bg = {
    "A": "Other Asian",
    "B": "Black",
    "C": "Chinese",
    "D": "Cambodian",
    "F": "Filipino",
    "G": "Guamanian",
    "H": "Hispanic/Latin/Mexican",
    "I": "American Indian/Alaskan Native",
    "J": "Japanese",
    "K": "Korean",
    "L": "Laotian",
    "O": "Other",
    "P": "Pacific Islander",
    "S": "Samoan",
    "U": "Hawaiian",
    "V": "Vietnamese",
    "W": "White",
    "X": "Unknown",
    "Z": "Asian Indian"
}
crimeData["Victim Descent"] = crimeData["Victim Descent"].map(Victims_bg)

In [9]:
crimeData.to_csv('./TableauReady/crimeData.csv', index=False)

In [10]:
children = ['Children and Family Services',
       'Early Childhoold Education and Head Start',
       'Private and Charter Schools', 'Child Care',
       'Community Organizations', 'Recreation Programs',
       'Special Curriculum Schools and Programs', 'Child Support Services',
       'Pools', 'Parks and Gardens', 'Libraries', 'Recreation Centers',
       'Museums and Aquariums', 'School Districts',
       'Cultural and Performing Arts Centers', 'Amtrak Stations',
       'Picnic Areas', 'Recreation Clubs', 'Public Middle Schools',
       'Public Elementary Schools', 'Metrolink Stations',
       'Park and Ride Locations', 'Metro Stations', 'Public High Schools',
       'Sports Complexes', 'Youth Worksource Centers']

poi = poi[poi['Category2'].isin(children)]

locations = list(poi['Location 1'])
lat = []
lon = []
address = []
for l in locations:
    try:
        address.append(l.split('(')[0])
        lat.append(np.float(l.split('(')[1].split(',')[0]))
        lon.append(np.float(l.split('(')[1].split(',')[1].lstrip().rstrip(')')))
    except:
        lat.append(None)
        lon.append(None)
poi['Address'] = address
poi['lat'] = lat
poi['lon'] = lon
poi['type'] = 'POI'
poi = poi.drop(['Location 1'], axis=1) #Delete not usable columns

In [11]:
CrimePOI = pd.concat([crimeData, poi])
CrimePOI.to_csv('./TableauReady/CrimePOI.csv', index=False)

In [12]:
CrimePOI.shape

(1709037, 34)